In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import sys
import pickle
import csv
import random
import re
from sklearn.model_selection import KFold

In [ ]:
#drug_iddict        final_Virus_DTI_add_organism.tsv  human_seq_dict.pkl    VDTI_net.npy           virusseq_add_ncov.pkl
#drug_info.tsv      HDTI_net.npy                      human_seq_iddict.pkl  VHI_net.npy            virusseq_add_ncovseqdict.pkl
#Drug_simi_net.npy  human.npy                         PPI_net.npy           virusseq_add_ncov.npy

In [ ]:
#NPY
drug_simi_net = np.load("DTI-data/Drug_simi_net.npy")
print(drug_simi_net.shape)
print(drug_simi_net[0])
HDTI_net = np.load("DTI-data/HDTI_net.npy")
print(HDTI_net.shape)
print(HDTI_net[0])
human = np.load("DTI-data/human.npy")
print(human.shape)
print(human[0])
ppi_net = np.load("DTI-data/PPI_net.npy")
print(ppi_net.shape)
print(ppi_net[0])
#This is our ultimate target
vdti_net = np.load("DTI-data/VDTI_net.npy")
print("!", vdti_net.shape)
print("!", vdti_net[0])
vhi_net = np.load("DTI-data/VHI_net.npy")
print(vhi_net.shape)
print(vhi_net[0])
virusseq_add_ncov = np.load("DTI-data/virusseq_add_ncov.npy")
print(virusseq_add_ncov.shape)
print(virusseq_add_ncov[0])

In [ ]:

#PICKLE
with open("DTI-data/human_seq_dict.pkl", 'rb') as f:
    human_seq_dict = pickle.load(f)
with open("DTI-data/human_seq_iddict.pkl", 'rb') as f:    
    human_seq_iddict = pickle.load(f)
with open("DTI-data/virusseq_add_ncov.pkl", 'rb') as f:
    virusseq_add_ncov_dict = pickle.load(f)
with open("DTI-data/virusseq_add_ncovseqdict.pkl", 'rb') as f:
    virusseq_add_ncovseqdict = pickle.load(f)
with open("DTI-data/drug_iddict", 'rb') as f:
    drug_iddict = pickle.load(f)

print(human_seq_dict[list(human_seq_dict.keys())[0]][:100], "\n")
print(human_seq_iddict[list(human_seq_iddict.keys())[0]], "\n")
print(virusseq_add_ncov_dict[list(virusseq_add_ncov_dict.keys())[0]], "\n")
print(virusseq_add_ncovseqdict[list(virusseq_add_ncovseqdict.keys())[0]][:100], "\n")
print(drug_iddict[list(drug_iddict.keys())[0]], "\n")

In [ ]:

#TSV
with open("DTI-data/drug_info.tsv", "r") as f:
    drug_info = csv.reader(f, delimiter = '\t')
    drug_info = list(drug_info)
with open("DTI-data/final_Virus_DTI_add_organism.tsv", "r") as f:
    final_Virus_DTI_add_organism = csv.reader(f, delimiter = '\t') 
    final_Virus_DTI_add_organism = list(final_Virus_DTI_add_organism)

print(drug_info[0:2])
print("")
print(final_Virus_DTI_add_organism[0:2])

In [ ]:
def prepare_data(iter=0, kfolds=10, seed=0):
    # KnowledgeGraphEmbedding expects 5 files

    # entities.dict - maps entities to ids
    # relations.dict - maps relations to ids
    # train.txt - entity id to relation id
    # valid.txt - can be blank
    # test.txt

    #print(HDTI_net.shape)
    #print(HDTI_net[0])

    #print("!", vdti_net.shape)
    #print("!", vdti_net[0])

    #print(vhi_net.shape)
    #print(vhi_net[0])

    #entities.dict

    DRUG_COUNT = 6255
    HUMAN_COUNT = 2567
    VIRUS_COUNT = 404

    sum_ = DRUG_COUNT + HUMAN_COUNT + VIRUS_COUNT

    HUMAN_OFFSET = 6255
    VIRUS_OFFSET = 6255 + 2567

    names = ["vdti", "hdti", "vhi", "ppi", "drug_s", "human_s", "virus_s"]
    vars = [vdti_net, HDTI_net, vhi_net, ppi_net, drug_simi_net, human, virusseq_add_ncov]
    lens = [len(arr) for arr in vars]
    #lens = [vdti_net.shape[0], HDTI_net.shape[0], vhi_net.shape[0], ppi_net.shape[0], drug_simi_net.shape[0],  uman_seq_dict.shape[0], virusseq_add_ncov.shape[0]]
    run_sum = [sum(lens[:i]) for i in range(1, len(names) + 1)]
    offset = {names[i]:run_sum[i] for i, _ in enumerate(names)}

    with open("KnowledgeGraphEmbedding/data/C-19/entities.dict", "w") as f:
        for i in range(DRUG_COUNT + HUMAN_COUNT + VIRUS_COUNT):
            f.write(str(i) + "\t" + str(i) + "\n")

    #relations.dict

    with open("KnowledgeGraphEmbedding/data/C-19/relations.dict", "w") as f:
        f.write("0\t0\n")
        f.write("1\t1\n")
        f.write("2\t2\n")
        f.write("3\t3\n")
        f.write("4\t4\n")
        f.write("5\t5\n")
        f.write("6\t6")
    #with open("KnowledgeGraphEmbedding/data/C-19/entities.dict" as f):

    #all.txt

    HIGH_SIM_THRES = 0.5

    with open("KnowledgeGraphEmbedding/data/C-19/all.dict", "w") as f:
        #drug -> human
        for (i, j), v in np.ndenumerate(HDTI_net):
            assert(j + HUMAN_OFFSET < sum_)
            if v:
                f.write(str(i) + "\t" + "0"  + "\t" + str(j + HUMAN_OFFSET) + "\n")

        #drug -> virus
        i = 0
        for (i, j), v in np.ndenumerate(vdti_net):
            if v:
                f.write(str(i) + "\t" + "1"  + "\t" +  str(j + VIRUS_OFFSET) + "\n")  

        #virus -> human
        i = 0
        for (i, j), v in np.ndenumerate(vhi_net):
            assert(i + VIRUS_OFFSET < sum_)
            assert(j + HUMAN_OFFSET < sum_)
            if v:
                f.write(str(i + VIRUS_OFFSET) + "\t" + "2"  + "\t" + str(j + HUMAN_OFFSET) + "\n")          

        for (i, j), v in np.ndenumerate(ppi_net):
            assert(i + HUMAN_OFFSET < sum_)
            assert(j + HUMAN_OFFSET < sum_)
            if v:
                f.write(str(i + HUMAN_OFFSET) + "\t" + "3"  + "\t" + str(j + HUMAN_OFFSET) + "\n")


        for (i, j), v in np.ndenumerate(drug_simi_net):
            assert(i  < sum_)
            assert(j  < sum_)
            if v >= HIGH_SIM_THRES:
                f.write(str(i) + "\t" + "4"  + "\t" + str(j) + "\n")


        for (i, j), v in np.ndenumerate(human):
            assert(i + HUMAN_OFFSET < sum_)
            assert(j + HUMAN_OFFSET < sum_)
            if v >= HIGH_SIM_THRES:
                f.write(str(i + HUMAN_OFFSET) + "\t" + "5"  + "\t" + str(j + HUMAN_OFFSET) + "\n")


        for (i, j), v in np.ndenumerate(virusseq_add_ncov):
            assert(i + VIRUS_OFFSET < sum_)
            assert(j + VIRUS_OFFSET < sum_)
            if v >= HIGH_SIM_THRES:
                f.write(str(i + VIRUS_OFFSET) + "\t" + "6"  + "\t" + str(j + VIRUS_OFFSET) + "\n")

    TRAIN_TEST_SPLIT = 0.85
    TRAIN_VAL_SPLIT  = 0.90

    print("start write")

    #TODO: rename drug variables to virus
    with open("KnowledgeGraphEmbedding/data/C-19/all.dict", "r") as f:
        lst = f.readlines()
        all_drug_indices = [i for i in range(VIRUS_OFFSET, VIRUS_COUNT + VIRUS_OFFSET)]
        kf = KFold(n_splits=kfolds, shuffle=True, random_state=seed)
        train_t_drug_indices,  test_drug_indices = list(kf.split(all_drug_indices))[iter]

        #kfold indices -> drug indices
        train_t_drug_indices = [all_drug_indices[i] for i in train_t_drug_indices]
        test_drug_indices    = [all_drug_indices[i] for i in test_drug_indices]

        """
        kf2 = KFold(n_splits=kfolds, shuffle=True, random_state=seed)
        list_of_tuple_of_list_of_indices = list(kf.split(train_t_drug_indices))
        train_drug_indices =  [tup[0] for tup in list_of_tuple_of_list_of_indices]
        val_drug_indices   =  [tup[1] for tup in list_of_tuple_of_list_of_indices]
        train_drug_indices =  [index for idxlist in train_drug_indices for index in idxlist]
        val_drug_indices   =  [index for idxlist in val_drug_indices   for index in idxlist]
        """

        #create validation set
        #remove validation data from train set
        val_drug_indices   = [i for i in train_t_drug_indices[::kfolds]]
        train_drug_indices = [i for i in train_t_drug_indices if i not in val_drug_indices]

        print("===")
        print("test indices", test_drug_indices)
        print("val indices", val_drug_indices)
        print("train indices sample", train_drug_indices[:100])
        print("===")

        assert(len(test_drug_indices) > 0 and len(train_t_drug_indices) > 0)
        #assert disjoint and span
        pass
        print(lst[0].split("\t"))
        print(lst[1].split("\t")[2].strip("\n"))

        test  = [e for e in lst if  int(e.split("\t")[2].strip("\n")) in test_drug_indices and e.split("\t")[1] in ["1"]]
        val   = [e for e in lst if  int(e.split("\t")[2].strip("\n")) in val_drug_indices  and e.split("\t")[1] in ["1"]]
        train = [e for e in lst if e not in test and e not in val]
        
        for e in [test, train, val]:
            assert(len(e) > 0)
        #train.txt
        with open("KnowledgeGraphEmbedding/data/C-19/train.txt", "w+") as f:
            f.writelines(train)
        #valid.txt
        with open("KnowledgeGraphEmbedding/data/C-19/valid.txt", "w+") as f:
            f.writelines(val) #val
        #test.txt
        with open("KnowledgeGraphEmbedding/data/C-19/test.txt",  "w+") as f:
            f.writelines(test)

    print("done")

In [ ]:
#train, test
# -lr 0.0001 --max_steps 150000 \
#    --do_valid \
kfolds = 10
seed   = 42

#Test gamma
#Reduce dimension
#Vary learning rate
#Max steps is less important
#Vary Modelee
#overwrites log each iteration!
#    -lr 0.0001 --max_steps 5000 \
for i in range(kfolds):
    prepare_data(i, kfolds, seed)

    if i % 4 == 0:
        !CUDA_VISIBLE_DEVICES=0 python -u KnowledgeGraphEmbedding/codes/run.py --do_train \
        --cuda \
        --do_test \
        --data_path KnowledgeGraphEmbedding/data/C-19 \
        --model RotatE \
        -n 256 -b 1024 -d 100 \
        -g 240.0 -a 2.0 -adv \
        -lr 0.0001 --max_steps 5000 \
        --n_targets 404 \
        --do_valid \
        --valid_steps 1000 \
        -save KnowledgeGraphEmbedding/models/RotatE_C-19_0 --test_batch_size 8 -de  2>&1| tee -a kge_log.txt

    elif i % 4 == 1:
        !CUDA_VISIBLE_DEVICES=0 python -u KnowledgeGraphEmbedding/codes/run.py --do_train \
        --cuda \
        --do_test \
        --data_path KnowledgeGraphEmbedding/data/C-19 \
        --model RotatE \
        -n 256 -b 1024 -d 100 \
        -g 24.0 -a 1.0 -adv \
        -lr 0.0001 --max_steps 5000 \
        --n_targets 404 \
        --do_valid \
        --valid_steps 1000 \
        -save KnowledgeGraphEmbedding/models/RotatE_C-19_0 --test_batch_size 8 -de  2>&1| tee -a kge_log.txt

    elif i % 4 == 2:
        !CUDA_VISIBLE_DEVICES=0 python -u KnowledgeGraphEmbedding/codes/run.py --do_train \
        --cuda \
        --do_test \
        --data_path KnowledgeGraphEmbedding/data/C-19 \
        --model RotatE \
        -n 256 -b 1024 -d 1000 \
        -g 3.0 -a 5.0 -adv \
        -lr 0.0001 --max_steps 5000 \
        --n_targets 404 \
        --do_valid \
        --valid_steps 1000 \
        -save KnowledgeGraphEmbedding/models/RotatE_C-19_0 --test_batch_size 8 -de  2>&1| tee -a kge_log.txt

    elif i % 4 == 3:
        !CUDA_VISIBLE_DEVICES=0 python -u KnowledgeGraphEmbedding/codes/run.py --do_train \
        --cuda \
        --do_test \
        --data_path KnowledgeGraphEmbedding/data/C-19 \
        --model RotatE \
        -n 256 -b 1024 -d 1000 \
        -g 96 -a 1.0 -adv \
        -lr 0.0001 --max_steps 20000 \
        --n_targets 404 \
        --do_valid \
        --valid_steps 1000 \
        -save KnowledgeGraphEmbedding/models/RotatE_C-19_0 --test_batch_size 8 -de  2>&1| tee -a kge_log.txt

In [ ]:
#capture output
with open("KnowledgeGraphEmbedding/models/RotatE_C-19_0/train.log") as f:
    output = f.readlines()
out = output
#print(out)
print(out)
res1 = [line for line in out if "auc_pr" in line]
res2 = [line for line in out if "auc_roc" in line]
#2020-05-29 10:58:05,858 INFO     Test auc_pr at step 9999: 0.817098
#2020-05-29 10:58:05,858 INFO     Test auc_roc at step 9999: 0.780551

#print(res)
#print(res1)
res1 = [re.findall("\d+\.\d+", r) for r in res1]
res2 = [re.findall("\d+\.\d+", r) for r in res2]

print(res1)
print(res2)

#deb = [line for line in out if "kfold" in line]

#print(deb)
#clear buffers
#output = ""
#out    = ""
#res    = ""

In [ ]:
OUT = """2020-05-30 17:28:46,433 INFO     Test auc_pr at step 19999: 0.716268
2020-05-30 17:28:46,433 INFO     Test auc_roc at step 19999: 0.682861
2020-05-30 19:05:18,762 INFO     Test auc_pr at step 19999: 0.759156
2020-05-30 19:05:18,762 INFO     Test auc_roc at step 19999: 0.706657
2020-05-30 20:41:21,922 INFO     Test auc_pr at step 19999: 0.739043
2020-05-30 20:41:21,922 INFO     Test auc_roc at step 19999: 0.647360
2020-05-30 22:17:48,734 INFO     Test auc_pr at step 19999: 0.728497
2020-05-30 22:17:48,734 INFO     Test auc_roc at step 19999: 0.640756
2020-05-30 23:56:06,703 INFO     Test auc_pr at step 19999: 0.676069
2020-05-30 23:56:06,703 INFO     Test auc_roc at step 19999: 0.655680
2020-05-31 01:34:04,867 INFO     Test auc_pr at step 19999: 0.645229
2020-05-31 01:34:04,867 INFO     Test auc_roc at step 19999: 0.603482
2020-05-31 03:11:51,151 INFO     Test auc_pr at step 19999: 0.736085
2020-05-31 03:11:51,151 INFO     Test auc_roc at step 19999: 0.701004
2020-05-31 04:48:40,105 INFO     Test auc_pr at step 19999: 0.730914
2020-05-31 04:48:40,105 INFO     Test auc_roc at step 19999: 0.734637
2020-05-31 06:25:25,569 INFO     Test auc_pr at step 19999: 0.802390
2020-05-31 06:25:25,569 INFO     Test auc_roc at step 19999: 0.758581
2020-05-31 08:02:56,224 INFO     Test auc_pr at step 19999: 0.787228
2020-05-31 08:02:56,224 INFO     Test auc_roc at step 19999: 0.762826
"""

res = re.findall("\d+.\d+\n", OUT)
auc_prs = [float(e.strip("\n")) for e in res[::2]]
auc_roc = [float(e.strip("\n")) for e in res[1::2]]

print(sum(auc_prs)/len(auc_prs))
print(sum(auc_roc)/len(auc_roc))